In [1]:
class Vowels:
    def __getitem__(self, i):
        return 'AEIOU'[i]

v = Vowels()
v[0]

'A'

In [2]:
v[-1]

'U'

In [3]:
for c in v: print(c)

A
E
I
O
U


In [4]:
'E' in v

True

In [5]:
'Z' in v

False

In [6]:
# if there is a __getitem__ python is smart enough to iterate (__iter__) if need to and even add __contains__ even if its missing (in operator)
# as long as its a sequence-like data structure

In [7]:
from random import shuffle
l = list(range(10))
shuffle(l)
l

[1, 9, 4, 2, 7, 3, 6, 5, 0, 8]

In [8]:
# GooseTyping
from collections import namedtuple, abc

Card = namedtuple('Card', ['rank', 'suit'])
class FrenchDeck2(abc.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

    # built in shuffle works by shuffling in place so this is all we need to use it
    def __setitem__(self, position, value):
        self._cards[position] = value

    # MutableSequence demands this:
    def __delitem__(self, position):
        del self._cards[position]

    # MutableSequence demands this:
    def insert(self, position, value):
        self._cards.insert(position, value)

In [9]:
# The only reliable way to determine an object is iterable is to call iter(obj)
# isinstance or issubclass could return False but the class could implement __getitem__ 0-based indices

In [29]:
# @abstractclassmethod, @abstractstaticmethod, and @abstractprop are deprecated!
import abc

class Tombola(abc.ABC):
    
    @abc.abstractmethod
    def load(self, iterable):
        """Add itemss from an iterable."""

    @abc.abstractmethod
    def pick(self):
        """Remove item at random, returning it.
        This method should raise `LookupError` when the instance is empty."""

    def loaded(self):
        """Returen `True` if there's at least 1 item, `False`, otherwise."""
        return bool(self.inspect())

    def inspect(self):
        """Return a sorted tuple with the items currenlty inside."""
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError: # LookupError -> IndexError or KeyError
                break;
        self.load(items)
        return tuple(items)

In [11]:
class Fake(Tombola):
    def pick(self):
        return 13

Fake

__main__.Fake

In [12]:
f = Fake() # throw

TypeError: Can't instantiate abstract class Fake with abstract method load

In [14]:
import random

class BingoCage(Tombola):
    def __init__(self, items):
        self._randomizer = random.SystemRandom() # uses os.urandom() random bytes suitable for cryptographic use
        self._items = []
        self.load(items)

    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)

    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')

    def __call__(self):
        self.pick()

In [16]:
import random

class LottoBlower(Tombola):
    def __init__(self, iterable):
        self._balls = list(iterable)

    def load(self, iterable):
        self._balls.extend(iterable)

    def pick(self):
        try:
            position = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError('pick from empty LottoBlower')
        return self._balls.pop(position)

    def loaded(self):
            return bool(self._balls)

    def inspect(self):
        return tuple(self._balls)

In [32]:
@Tombola.register
class TomboList(list):
    load = list.extend

    def pick(self):
        if self:
            position = random.randrange(len(self))
            return self.pop(position)
        else:
            raise LookupError('pop from empty TomboList')

    def loaded(self):
        return bool(self)
        
    def inspect(self):
        return tuple(self)

# @Tambola.register(TombaList), useful to register a class we don't maintain but fulfills the interface

In [33]:
issubclass(TomboList, Tombola)

True

In [34]:
t = TomboList(range(100))
isinstance(t, Tombola)

True

In [36]:
TomboList.__mro__ # method resolution order

(__main__.TomboList, list, object)

In [37]:
# __subclass__ hook determines issubclass, isinstance
# for example if _len__ if implemented in class:
class AnswerToEverything:
    def __len__(self): return 42

from collections import abc
isinstance(AnswerToEverything(), abc.Sized)

True

In [38]:
issubclass(AnswerToEverything, abc.Sized)

True

In [39]:
"""
Under the hood implementation of Sized:
class Sized(metaclass=ABCMeta):
    __slots__ = ()

    @abstractmethod
    def __len__(self):
        return 0

    @classmethod
    def __subclasshook__(cls, C):
        if cls is Sized:
            if any('__len__' in B.__dict__ for B in C.__mro__):
                return True
        return NotImplemented
"""

"\nUnder the hood implementation of Sized:\nclass Sized(metaclass=ABCMeta):\n    __slots__ = ()\n\n    @abstractmethod\n    def __len__(self):\n        return 0\n\n    @classmethod\n    def __subclasshook__(cls, C):\n        if cls is Sized:\n            if any('__len__' in B.__dict__ for B in C.__mro__):\n                return True\n        return NotImplemented\n"

In [40]:
# __len__, __getitem__, and __iter__ are not consider subtypes of Sequence because we can’t retrieve items using integer offsets or slices

In [41]:
# c64 = np.complex64(3+4j) is one of five numpy complex types
# test an object c is a complex or SupportsComplex:
# isinstance(c, (complex, SupportsComplex)) where c is the variable (c = complex(c64))

In [42]:
import sys
sys.version

'3.10.15 | packaged by conda-forge | (main, Oct 16 2024, 01:24:20) [Clang 17.0.6 ]'

In [46]:
# This is False and fixed in our current version of python
from typing import SupportsFloat
c = 3+4j
isinstance(c, SupportsFloat)

False

In [48]:
from typing import Protocol, runtime_checkable, Any

@runtime_checkable
class RandomPicker(Protocol):
    def pick(self) -> Any: ...

In [50]:
import random
from typing import Any, Iterable, TYPE_CHECKING

class SimplePicker:
    def __init__(self, items: Iterable) -> None:
        self._items = list(items)
        random.shuffle(self._items)

    def pick(self) -> Any:
        return self._items.pop()

def test_isinstance() -> None:
    popper: RandomPicker = SimplePicker([1])
    assert isinstance(popper, RandomPicker) # works because @runtime_checkable, and SimplePicker has a pick()

def test_item_type() -> None:
    items = [1,2]
    popper = SimplePicker(items)
    item = popper.pick()
    assert item in items
    if TYPE_CHECKING:
        reveal_type(item)
    assert isinstance(item, int)

In [51]:
# Narrow protocols are favorable, single methods or a couple
# Clients should not be forced to depend on interfaces they do not use

In [52]:
# extending randompicker
class LoadableRandomPicker(RandomPicker, Protocol):
    def load(self, Iterable) -> None: ...

In [54]:
"""
decimal.Decimal is not registered as a subclass of numbers.Real.
if you need the precision of Decimal in your program, then you want to be pro‐
tected from accidental mixing of decimals with floating-point numbers that are less precise.
"""

'\ndecimal.Decimal is not registered as a subclass of numbers.Real.\nif you need the precision of Decimal in your program, then you want to be pro‐\ntected from accidental mixing of decimals with floating-point numbers that are less precise.\n'

In [55]:
"""
The numeric static protocols SupportsComplex, SupportsFloat, etc. work well
for static typing, but are unreliable for runtime type checking when complex
numbers are involved.
"""

'\nThe numeric static protocols SupportsComplex, SupportsFloat, etc. work well\nfor static typing, but are unreliable for runtime type checking when complex\nnumbers are involved.\n'